In [330]:
import logging
import sys
from re import *
import networkx as nx
import os
import re
import csv
import string

In [331]:
logger = logging.Logger('catch_all')

remultword = compile('\d+.\d+')

def read_graph_from_lines(graph_lines: list) -> nx.DiGraph:
    di_graph = nx.DiGraph()
    di_graph.add_node(0)
    for line in graph_lines:
        #print(line)
        di_graph.add_node(
            int(line.split('\t')[0]),
            token=line.split('\t')[1],
            lemma=line.split('\t')[2].strip(),
            upos=line.split('\t')[3],
            xpos=line.split('\t')[4],
            ufeats=line.split('\t')[5],
            misc=line.split('\t')[9])
        di_graph.add_edge(
            int(line.split('\t')[6]),
            int(line.split('\t')[0]),
            ud_label=line.split('\t')[7])
    return di_graph


def read_graph(conllu, howManyGraphs=3000):
    digraphs = []
    newdoc_ids = []
    doc_sentences_count = []
    last_sent_id = []
    k = 0
    try:
        graph_lines = []
        i = 0
        j = 0
        for line in conllu.readlines():
            if line.strip():
                if line.startswith('# '):
                    if line.startswith('# newdoc id'):
                        if last_sent_id != []:
                            doc_sentences_count.append(int(last_sent_id[0].split('-')[1]))
                        newdoc_ids.append(line.split()[4:][0])

                    #print(line
                    elif line.startswith('# sent_id'):
                        last_sent_id = line.split()[3:]
                        #print(line.split()[3:])
                    pass
                elif remultword.search(line.split()[0]):
                    if 'multiword_tokens' not in locals():
                        multiword_tokens = []
                    multiword_tokens.append(line.strip())
                else:
                    graph_lines.append(line.strip())
                    j += 1
            else:
                di_graph = read_graph_from_lines(graph_lines[k:k+j])
                digraphs.append((di_graph, multiword_tokens if 'multiword_tokens' in locals() else []))
                if 'multiword_tokens' in locals():
                    del multiword_tokens
                else:
                    None
                k += j
                j = 0
                i = i + 1
            if i == howManyGraphs:
                break
        doc_sentences_count.append(int(last_sent_id[0].split('-')[1]))

    except Exception as e:
        logger.error(e, exc_info=True)
        return []
    return digraphs, newdoc_ids, doc_sentences_count



if __name__ == '__main__':
    try:
        conllu = open('eng.erst.gum_dev.conllu', 'r', encoding="utf8")
    except IOError:
        print("The input conllu file not found")
        sys.exit(1)
    except IndexError:
        print("python", sys.argv[0], "inputfile outputfile")
        sys.exit(1)
    graphs, newdoc_ids, doc_sentences_count = read_graph(conllu, 150)

In [332]:

def getGovernor(graph, node):
   return nx.predecessor(graph, 0)[node]


print(graphs[12][0].nodes[1]['upos'])

#for node in graphs[12][0].nodes(data={"token"}):
 # print(node)

DET


In [333]:
TAGS = { 'acl:relcl', 'root'}
VERB_TAGS = {'parataxis', 'acl', 'acl:relcl', 'advcl'}
DISCOURSE_MARKERS = {'and', 'despite', 'prior'}

#NOT USED YET
def checkForDiscourseMarkers(graph, token):
    found = False
    for successor in graph.successors(token):
        if graph.nodes[successor]['lemma'] in DISCOURSE_MARKERS and getGovernor(graph, getGovernor(graph, getGovernor(graph, successor)[0])[0])[0] != token-1:
            found = True
            break
    return found
##


def find_governors_from_graph(graph: nx.DiGraph, subroot=None) -> list:
    def is_valid_governor(root, token):
        upos = graph.nodes[token]['upos']
        deprel = graph.edges[root, token]['ud_label']
        root_upos = graph.nodes[root]['upos']
        pron_type = graph.nodes[token]['ufeats']
        #print(graph.nodes[token]['token'])
        #1. ogólna zasada, ale z dość szczegółowym wyjątkiem
        if deprel in TAGS:
            if pron_type == "PronType=Rel" and root_upos == "PRON":
                return False
            #if deprel == "acl:relcl" and graph.edges[getGovernor(graph, root)[0], root]["ud_label"] == "nmod" and root_upos == "PRON":
             #   return False
            return True

        #2. ogólna
        elif upos == "VERB" and deprel in VERB_TAGS:
            return True

        #3. dość ogólna
        elif upos == "VERB" and deprel == "conj" and graph.edges[getGovernor(graph, root)[0], root]["ud_label"] not in {"xcomp"}: #acl:
            return True

        #4. dość ogólna
        elif upos == "VERB" and deprel == "ccomp":# and check_successors(graph, token, {"SCONJ"}, {"mark"})[0]:
            return True
        #5. ogólna
        elif upos == "ADJ" and deprel in VERB_TAGS and root_upos not in {"ADJ", "NOUN", "PRON"}:
            return True
        #6. dość ogólna
        elif upos == "ADV" and deprel == "advcl:relcl":# and check_successors(graph, token, {"VERB"}, {"xcomp", "aux"})[0]:
            return True

        #elif upos == "VERB" and deprel == "advcl:relcl" and graph.nodes[root]['ufeats'] == "PronType=Rel":
         #   return True


        #7. raczej szczegółowa
        elif upos == "ADV" and deprel == "obl" and pron_type == "PronType=Rel" and graph.edges[getGovernor(graph, root)[0], root]["ud_label"] == "xcomp":
            return True

        #13. szczegółowa: wydzielanie nawiasów i ich zawartości
        elif deprel == "dep" and check_successors(graph, token, {"PUNCT"}, {"punct"})[0]:
             for successor in graph.successors(token):
                if graph.nodes[successor]['lemma'] in {"(", "{"}:
                    for successor2 in graph.successors(token):
                        if graph.nodes[successor2]['lemma'] in {")", "}"}:
                            return True
        #14 ja bym wydzielał tylko jeśli wśród rodziców nie ma już tokenu oznaczonego jako gov
        elif deprel == "discourse":
            return True

        #9. raczej szczegółowa

        #elif upos == "NOUN" and root_upos != "NOUN" and check_successors(graph, token, {"AUX"}, {"cop"}) and check_successors(graph, token, {"CCONJ"}, {"cc"})[0]:
         #   return True

        '''
        #8. raczej szczegółowa
        elif upos == "NOUN" and deprel == "appos" and check_successors(graph, token, {"PUNCT"}, {"punct"})[0]:
            return True'''



        '''
        #10. szczegółowa: liczby w nawiasach bo dużo ich było w tekście naukowym jako przypisy
        elif upos == "NUM" and deprel in {"nmod", "dep"}:
            first_gov = getGovernor(graph, token)[0]
            if first_gov == 0:
                return False
            second_gov = getGovernor(graph, first_gov)[0]
            second_deprel = graph.edges[second_gov, first_gov]['ud_label']
            if second_deprel not in {"nmod", "dep", "conj", "nummod"}:
                return True
            elif graph.nodes[first_gov]['upos'] != "NUM":
                return True
        #elif upos == "PROPN" and deprel == "conj" and root_upos == "PROPN" and root != 0 and graph.edges[getGovernor(graph, root)[0], root]["ud_label"] == "dep":
         #   return True
        #11.
        elif deprel == "nmod" and check_successors(graph, token, {"VERB"}, {"case"})[0]:
            return True
            '''#elif upos == "SCONJ" and deprel == "mark":
            #gov_token = getGovernor(graph, token)[0]
            #if token + 1 != gov_token:
             #   return True
        '''

        #12. WYDZIELANIE FRAZ PRZYIKMOWYCH Z MARKERAMI DYSKURSU
        elif deprel == "obl":
           successor = check_successors(graph, token, {"ADP"}, {"case"})[0]
           if successor and successor['lemma'] in DISCOURSE_MARKERS:
                print("ZNALEZION")
                return True
           else:
                successor = check_successors(graph, token, {"ADV"}, {"case"})
                if successor[1] == 0:
                    return False
                elif successor and check_successors(graph, successor[1], {"ADP"}, {"fixed"}) and successor[0]['lemma'] in DISCOURSE_MARKERS:
                    return True
        '''


        return False

    root = subroot if subroot is not None else list(graph.successors(0))[0]
    governors = [root] if subroot is None else []

    for token in graph.successors(root):
        if is_valid_governor(root, token):
            governors.append(token)
        governors.extend(find_governors_from_graph(graph, token))


    return governors


In [334]:

def check_successors(graph, token, target_upostags, target_deprels):
    """
    Check if any successors of a given token match specific UPOS tags and dependency relations.
    Returns the matching token's node attributes if found, otherwise False.
    """
    for successor in graph.successors(token):
        upos = graph.nodes[successor]['upos']
        governor = getGovernor(graph, successor)[0]
        deprel = graph.edges[governor, successor]['ud_label']

        if upos in target_upostags and deprel in target_deprels:
            return graph.nodes[successor], successor

    return False, False


In [335]:
def get_spans(graph: nx.DiGraph, span_heads: list, idx) -> list:
    """
    Given a list of head nodes, return non-overlapping descendant spans from the graph.

    Args:
        graph: A NetworkX DiGraph representing the dependency structure.
        span_heads: A list of node IDs considered as span heads.

    Returns:
        A list of sets, each representing a proper (non-overlapping) span.
    """
    all_spans = []

    # Collect full descendant spans including the head
    for head in span_heads:
        span = nx.descendants(graph, head)
        span.add(head)
        span &= set(graph.nodes)  # Ensure span only includes valid node IDs
        all_spans.append(span)

    # Sort spans by size (largest first)
    all_spans.sort(key=len, reverse=True)

    proper_spans = []
    for i, current_span in enumerate(all_spans):
        # Remove overlap with all smaller spans
        for other_span in all_spans[i+1:]:
            current_span -= other_span
        if current_span:
            proper_spans.append(current_span)

    # Sort spans by their minimum node ID for consistency
    proper_spans.sort(key=min)
    if idx == GLOBAL:
        print(proper_spans)
    return proper_spans


def check_continuity(my_list):
    return all(a+1==b for a, b in zip(my_list, my_list[1:]))


def findSplit(my_list):
    splits = []
    for a, b in zip(my_list, my_list[1:]):
        if a+1!=b:
            splits.append((a, b))
    return splits

def removeDiscourseFromGovs(graph, governors):
    new_governors = []
    for gov in governors:
        if graph.edges[getGovernor(graph, gov)[0], gov]['ud_label'] != "discourse":
            new_governors.append(gov)
        elif getGovernor(graph, gov)[0] not in governors:
            new_governors.append(gov)
            print("I DO")
    return new_governors

def prepare_doc(graph: nx.DiGraph, idx) -> list:
    """
    Prepares a list of beginning token indices for continuous spans found in the graph.

    Returns:
        List of token indices marking the beginning of each proper span.
    """
    governors = find_governors_from_graph(graph, None)
    governors = removeDiscourseFromGovs(graph, governors)

    if idx == GLOBAL:
      print(f"gov {sorted(governors)}")
    # Unpack both spans and discourse markers (if present)
    token_spans = get_spans(graph, governors, idx)

    proper_spans = []

    for raw_span in token_spans:
        sorted_span = sorted(raw_span)

        if check_continuity(sorted_span):
            proper_spans.append(sorted_span)
        else:
            splits = findSplit(sorted_span)

            if splits:
                for i, (start, end) in enumerate(splits):
                    start_idx = sorted_span.index(start)
                    end_idx = sorted_span.index(end)

                    if i == 0:
                        proper_spans.append(sorted_span[:start_idx + 1])
                        if len(splits) == 1:
                            proper_spans.append(sorted_span[end_idx:])
                    elif i < len(splits) - 1:
                        prev_end_idx = sorted_span.index(splits[i - 1][1])
                        proper_spans.append(sorted_span[prev_end_idx:start_idx + 1])
                    else:
                        prev_end_idx = sorted_span.index(splits[i - 1][1])
                        proper_spans.append(sorted_span[prev_end_idx:start_idx + 1])
                        proper_spans.append(sorted_span[end_idx:])

    # Remove invalid span if it accidentally includes a non-existent index
    node_count = len(graph)
    if [node_count] in proper_spans:
        proper_spans.remove([node_count])

    # Return the starting index of each proper span
    span_beginnings = [span[0] for span in proper_spans]
    return span_beginnings

In [336]:
def prepare_data(discourse_unit_starts: list, texts: list) -> list:
    """
    Generates segmentation labels for each token in the input texts.

    Args:
        discourse_unit_starts: A list of lists, where each inner list contains indices
                               marking the beginning of a discourse unit for a corresponding text.
        texts: A list of tokenized texts (as lists of tokens or characters).

    Returns:
        A list of lists containing segmentation labels ("Seg=B-seg" or "Seg=O") for each token.
    """
    eduLabels = []

    for tokens, start_indices in zip(texts, discourse_unit_starts):
        if tokens and not tokens[0].startswith("# "):  # skip comment lines
            labels = ["Seg=O"] * len(tokens)
            for idx in start_indices:
                if 0 <= idx - 1 < len(labels):
                    labels[idx - 1] = "Seg=B-seg"
            eduLabels.append(labels)
        else:
            eduLabels.append([None])
    return eduLabels

In [337]:
def main():
    # Load the input file and parse it into graphs
    with open('eng.erst.gum_dev.conllu', 'r', encoding="utf8") as conllu_file:
        graphs, newdoc_ids, doc_sentences_count = read_graph(conllu_file)

    token_lists = []
    predicted_beginnings = []
    #############################
    if False:
        for graph in graphs[idx:idx+1]:
            for node in graph[0].nodes:
                print(graph[0].nodes[node])



    #########################
    mwt_tab = []
    ellipsis_tab = []
    for i, (graph, mwt_lines) in enumerate(graphs):
      # Parse MWT lines into a dict for quick lookup
        # Prepare MWTs as a dictionary
        mwt_dict = {}
        ellipsis_dict = {}
        for line in mwt_lines:
            parts = line.split('\t')
            if '-' in parts[0]:
                mwt_dict[int(parts[0].split('-')[0])] = (parts[0], parts[1])  # {893: ("893-894", "that's")}
            elif '.' in parts[0]:
                ellipsis_dict[int(parts[0].split('.')[0])] = (parts[0], parts[1])
        mwt_tab.append(mwt_dict)
        ellipsis_tab.append(ellipsis_dict)

        tokens = [graph.nodes[node]['token'] for node in sorted(graph.nodes) if node != 0]
        token_lists.append(tokens)
        beginnings = prepare_doc(graph, i)
        if i == GLOBAL:
            print(f"BEG {sorted(beginnings)}")
            print(tokens)


        # Post-process beginnings to adjust for punctuation and edge cases
        clean_beginnings = []
        for idx in beginnings:
            token = graph.nodes[idx]['token']
            if token not in {",", "’", '"', "”", "“", ":", ";", ")"}:
                clean_beginnings.append(idx)
            elif idx + 1 < len(graph.nodes):  # Shift forward if punctuation
                clean_beginnings.append(idx + 1)

        # Remove final token as EDU if it's just the last token
        if (len(graph.nodes) - 1) in clean_beginnings and len(graph.nodes) > 2:
            clean_beginnings.remove(len(graph.nodes) - 1)

        clean_beginnings = list(set(clean_beginnings))
        if i == GLOBAL:
            print(f"CLEAN {sorted(clean_beginnings)}")
        predicted_beginnings.append(clean_beginnings)

    # Create EDU segmentation tags
    edu_labels = prepare_data(predicted_beginnings, token_lists)

    # Write the output in tokenized format
    with open('predicted.tok', "w", encoding="utf8", newline="") as out_file:
        writer = csv.writer(out_file, delimiter="\t")
        sent_index = 1

        all_sentence_counter = 0

        for doc_index, doc_id in enumerate(newdoc_ids):
            token_idx = 1
            if doc_index > 0:
                writer.writerow([])  # Separate documents with a blank line

            writer.writerow([f'# newdoc id = {doc_id}'])

            # Get sentence span for current document
            doc_start = sum(doc_sentences_count[:doc_index])
            doc_end = doc_start + doc_sentences_count[doc_index]
            doc_data = zip(token_lists[doc_start:doc_end], edu_labels[doc_start:doc_end])


            for i, (token_seq, label_seq) in enumerate(doc_data):
                token_in_sentence_counter = 1
                for token, label in zip(token_seq, label_seq):


                    # Check if this token is the start of a multi-word token
                    if token_in_sentence_counter in mwt_tab[sent_index-1]:
                        _, mwt_form = mwt_tab[sent_index-1][token_in_sentence_counter]
                        writer.writerow([f"{token_idx}-{token_idx+1}", mwt_form] + ['_'] * 8)

                    if token == '"':
                        out_file.write(f"{token_idx}\t{token}\t_\t_\t_\t_\t_\t_\t_\t{label}\n")

                    else:
                        row = [token_idx, token, '_', '_', '_', '_', '_', '_', '_', label]
                        writer.writerow(row)
                    if token_in_sentence_counter in ellipsis_tab[sent_index-1]:
                       elided_idx, elided_form = ellipsis_tab[sent_index-1][token_in_sentence_counter]
                       writer.writerow([f"{token_idx}.{elided_idx.split('.')[1]}", elided_form] + ['_'] * 8)

                    token_idx += 1
                    token_in_sentence_counter += 1

                sent_index += 1

    #print(f"lol: {[(i, tab) for (i, tab) in enumerate(mwt_tab) if tab != {}]}")

GLOBAL = 114

main()

gov [8, 10]
[{1, 2, 3, 4, 5, 6, 7, 8, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47}, {9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}]
BEG [1, 9, 25]
['NOW', ',', 'THEREFORE', ',', 'we', 'do', 'hereby', 'Order', 'and', 'Direct', 'Major', '-', 'General', 'Scott', ',', 'the', 'Command', '-', 'in', '-', 'Chief', 'of', 'our', 'Armies', ',', 'immediately', 'upon', 'receipt', 'of', 'this', ',', 'our', 'Decree', ',', 'to', 'proceed', 'with', 'a', 'suitable', 'force', 'and', 'clear', 'the', 'Halls', 'of', 'Congress', '.']
CLEAN [1, 9, 26]
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
I DO
